In [2]:
import numpy as np
import pandas as pd
from scipy import stats

import matplotlib
#%matplotlib notebook
%matplotlib qt5
#%matplotlib inline
from matplotlib import pyplot as plt

import seaborn as sns

matplotlib.rcParams['figure.figsize'] = (20.0, 10.0) # bigger figure!


sns.set() # better looking figs


In [3]:
# our own set of small helper functions for plotting, etc
from utils import plot_embedding, plot_compare_embeddings, show_heatmap, plot_confusion_matrix

In [4]:
df = pd.read_csv("full-data.csv")
df["clipId"] = df["clipName"].apply(lambda x: x[-8:-6])

# re-order columns + keep only useful ones
df = df[['pptID',
 'fileName',
 'condition',
 'age',
 'gender',
 'nationality',
 'firstLang',
 'trial',
 'clipId',
 'freetext',
 'q01',
 'q02',
 'q03',
 'q04',
 'q05',
 'q06',
 'q07',
 'q08',
 'q09',
 'q10',
 'q11',
 'q12',
 'q13',
 'q14',
 'q15',
 'q16',
 'q17',
 'q18',
 'q19',
 'q20',
 'q21',
 'q22',
 'q23',
 'q24',
 'q25',
 'q26',
 'q27',
 'q28',
 'q29',
 'q30']]

df

,pptID,fileName,condition,age,gender,nationality,firstLang,trial,clipId,freetext,...,q21,q22,q23,q24,q25,q26,q27,q28,q29,q30
0,186,10kin7dm75u231_data.csv,2,30,Female,Indian,Tamil,1,04,PLAY TOGETHER THE CARD GAME TO ENJOY THEM . IN...,...,2,2,1,2,1,2,2,2,2,2
1,186,10kin7dm75u231_data.csv,2,30,Female,Indian,Tamil,2,09,PLAYING TOGETHER FOR TWO CHILDREN . TO PLAY CA...,...,3,4,3,3,3,4,3,4,3,3
2,186,10kin7dm75u231_data.csv,2,30,Female,Indian,Tamil,3,16,PLAYING TOGETHER THE GAME. PLAY TO LEARNING TH...,...,3,4,3,3,3,3,3,4,3,3
3,186,10kin7dm75u231_data.csv,2,30,Female,Indian,Tamil,4,02,PLAYING TO LEARNING TOGETHER. TO PLAY INTEREST...,...,3,4,3,3,2,2,3,3,3,3
4,94,10kinos8b34va6_data.csv,1,23,Male,American,English,1,15,I notice that they slow down towards the end m...,...,1,1,2,1,1,2,2,2,2,2
5,94,10kinos8b34va6_data.csv,1,23,Male,American,English,2,02,The child on the right seems to be taking over...,...,1,0,1,1,1,3,4,1,3,0
6,94,10kinos8b34va6_data.csv,1,23,Male,American,English,3,08,"Both seem pretty calm, but the child on the le...",...,4,1,4,1,2,1,1,2,3,2
7,94,10kinos8b34va6_data.csv,1,23,Male,American,English,4,11,These children seem to be working together pre...,...,1,1,1,1,1,1,2,2,1,1
8,155,10kinqv5zq7rl5_data.csv,2,28,Male,American,English,1,12,They got along quite well and helped each othe...,...,0,4,3,2,2,4,2,2,4,2
9,155,10kinqv5zq7rl5_data.csv,2,28,Male,American,English,2,07,They played separately largely. They were fine...,...,3,0,4,1,3,4,3,3,0,2


Rename `qXX` columns with the names of the actual constructs

In [5]:
constructs=["Sad", "Happy", "Angry", "Excited", "Calm", "Friendly", "Aggressive", "Engaged", "Distracted", "Bored", "Frustrated","Dominant","Submissive"]

index = df.columns.tolist()
index = index[0:10] + ["Competing", "Cooperating", "PlaySeparate", "PlayTogether"] + [c for c1 in constructs for c in ['left' + c1, 'right' + c1]]
df.columns=index
df

,pptID,fileName,condition,age,gender,nationality,firstLang,trial,clipId,freetext,...,leftDistracted,rightDistracted,leftBored,rightBored,leftFrustrated,rightFrustrated,leftDominant,rightDominant,leftSubmissive,rightSubmissive
0,186,10kin7dm75u231_data.csv,2,30,Female,Indian,Tamil,1,04,PLAY TOGETHER THE CARD GAME TO ENJOY THEM . IN...,...,2,2,1,2,1,2,2,2,2,2
1,186,10kin7dm75u231_data.csv,2,30,Female,Indian,Tamil,2,09,PLAYING TOGETHER FOR TWO CHILDREN . TO PLAY CA...,...,3,4,3,3,3,4,3,4,3,3
2,186,10kin7dm75u231_data.csv,2,30,Female,Indian,Tamil,3,16,PLAYING TOGETHER THE GAME. PLAY TO LEARNING TH...,...,3,4,3,3,3,3,3,4,3,3
3,186,10kin7dm75u231_data.csv,2,30,Female,Indian,Tamil,4,02,PLAYING TO LEARNING TOGETHER. TO PLAY INTEREST...,...,3,4,3,3,2,2,3,3,3,3
4,94,10kinos8b34va6_data.csv,1,23,Male,American,English,1,15,I notice that they slow down towards the end m...,...,1,1,2,1,1,2,2,2,2,2
5,94,10kinos8b34va6_data.csv,1,23,Male,American,English,2,02,The child on the right seems to be taking over...,...,1,0,1,1,1,3,4,1,3,0
6,94,10kinos8b34va6_data.csv,1,23,Male,American,English,3,08,"Both seem pretty calm, but the child on the le...",...,4,1,4,1,2,1,1,2,3,2
7,94,10kinos8b34va6_data.csv,1,23,Male,American,English,4,11,These children seem to be working together pre...,...,1,1,1,1,1,1,2,2,1,1
8,155,10kinqv5zq7rl5_data.csv,2,28,Male,American,English,1,12,They got along quite well and helped each othe...,...,0,4,3,2,2,4,2,2,4,2
9,155,10kinqv5zq7rl5_data.csv,2,28,Male,American,English,2,07,They played separately largely. They were fine...,...,3,0,4,1,3,4,3,3,0,2


Compute, for each left/right constructs, the differences and sums. This gives us an insight on the inbalance of the given construct between the children, and the overall 'magnitude' of the construct in the clip.

In [6]:
for c in constructs:
    df["diff"+c] = abs(df["left" + c] - df["right" + c])
    df["sum"+c] = df["left" + c] + df["right" + c] - 4
    
# create 2 lists of columns names, one for left/right constructs, one for diff/sum constructs
columnsLeftRight=[]
columnsDiffSum=[]

for c in constructs:
    columnsLeftRight.append("left" + c)
    columnsLeftRight.append("right" + c)
    
    columnsDiffSum.append("diff" + c)
    columnsDiffSum.append("sum" + c)
    

df[df["condition"]==2].to_csv("data_fullscene.csv")
df[df["condition"]==1].to_csv("data_skel.csv")

# work with differences & sum for each constructs
selectedColumns=columnsDiffSum

## work with left child/right child for each constructs
#selectedColumns=columnsLeftRight

allQuestionsDiffSum = ["Competing", "Cooperating", "PlaySeparate", "PlayTogether"] + columnsDiffSum
allQuestionsLeftRight = ["Competing", "Cooperating", "PlaySeparate", "PlayTogether"] + columnsLeftRight
allQuestions = ["Competing", "Cooperating", "PlaySeparate", "PlayTogether"] + selectedColumns


*we define here several useful partial views of the main dataframe*

In [7]:
fullscene_df=df[df["condition"]==2] # full scene

# the responses to the 26 left/right Likert-scale questions
fullscene_ratings_df=fullscene_df[selectedColumns].astype(float)
fullscene=fullscene_ratings_df.values # the underlying numpy array

# clip names
fullscene_labels=fullscene_df["clipId"].values

# mean ratings per clip
fullscene_means=fullscene_df.groupby(["clipId"]).mean()[selectedColumns]


skel_df=df[df["condition"]==1] # skeleton

# the responses to the 26 left/right Likert-scale questions
skel_ratings_df=skel_df[selectedColumns].astype(float)
skel=skel_ratings_df.values # the underlying numpy array

# clip names
skel_labels=skel_df["clipId"].values

# mean ratings per clip
skel_means=skel_df.groupby(["clipId"]).mean()[selectedColumns]

In [8]:
meanvar_full_ratings=fullscene_df.groupby(["clipId"]).std()[columnsLeftRight].T.mean()
meanvar_skel_ratings=skel_df.groupby(["clipId"]).std()[columnsLeftRight].T.mean()

pd.DataFrame([meanvar_full_ratings,meanvar_skel_ratings],index=["mean stddev fullscene ratings","mean stddev skel ratings"]).T

,mean stddev fullscene ratings,mean stddev skel ratings
clipId,,
01,0.943597,1.068177
02,0.918407,0.984399
03,1.069666,0.916187
04,1.028272,1.056938
05,1.016512,0.936117
06,1.009450,0.939598
07,1.087390,1.037589
08,1.046094,0.956287
09,1.044294,1.018345


In [9]:
import krippendorff

krip={}

for clipName, group in fullscene_df[["clipId"] + allQuestionsLeftRight].groupby(["clipId"]):
    krip[clipName]=(krippendorff.alpha(group.values[:,1:].astype(int),level_of_measurement='interval'), group.shape[0])

for clipName, group in skel_df[["clipId"] + allQuestionsLeftRight].groupby("clipId"):
    krip[clipName]=krip[clipName] + (krippendorff.alpha(group.values[:,1:].astype(int),level_of_measurement='interval'), group.shape[0])

    
krippendorff_df=pd.DataFrame.from_dict(krip,orient="index", columns=["alpha(fullscene)", "N(fullscene)", "alpha(skel)", "N(skel)"])

show_heatmap(krippendorff_df[["alpha(fullscene)", "alpha(skel)"]], cmap="summer")




,alpha(fullscene),alpha(skel)
01,0.508508,0.186386
02,0.19098,0.269889
03,0.392881,0.368891
04,0.465137,0.26173
05,0.327889,0.283435
06,0.46281,0.359461
07,0.0914201,0.235908
08,0.339198,0.311564
09,0.0974896,0.0578229
10,0.396426,0.0861424


Comparing the agreement scores in the fullscene vs skeleton-only videos, using a T-test:

In [10]:
from scipy.stats import ttest_rel

fullscene_krip = krippendorff_df["alpha(fullscene)"]
skel_krip = krippendorff_df["alpha(skel)"]

ttest_rel(fullscene_krip, skel_krip)

Ttest_relResult(statistic=2.8691538900239397, pvalue=0.009821531836503023)

# Latent constructs

The first step of the analysis looks at latent constructs.

Our initial data contains responses to 22 questions (ie, 22 degrees of freedom). The question is: can those 22 DoFs be grouped into a smaller number of *latent* constructs that would effectively encapsulate the differences observed in the reponses between video clips.

Three approaches are explored:
- Principal component analysis (PCA)
- Principal component analysis (purely for dimensionality reduction) followed by a linear discriminant analysis (LDA) that aims at maximising inter-class distances (ie, inter-clips ratings) while minimizing intra-class distances (ie, the differences between ratings for a given clip).
- Explorative Factor Analysis (EFA)

## PCA

In [14]:
from sklearn.decomposition import PCA

We compute the PCA transformation with the responses to the *fullscene* stimuli.

We then project both the *fullscene* and the *skeletal-only* responses in this PCA space, effectively reducing the dimensionality of our data from 22 to `nb_components` (ie, 6).

In [15]:
nb_components = 6

fullscene_pca_model=PCA(n_components=nb_components).fit(fullscene)

fullscene_pca = fullscene_pca_model.transform(fullscene)
fullscene_means_pca = fullscene_pca_model.transform(fullscene_means.values)

skel_pca_model = fullscene_pca_model

skel_pca = skel_pca_model.transform(skel)
skel_means_pca = skel_pca_model.transform(skel_means.values)

With 6 components, about 70% of the variance in the *fullscene* dataset is explained.

In [16]:
pd.DataFrame(fullscene_pca_model.explained_variance_ratio_).plot(ylim=[0,1])
print("Cumulative explained variance: %s" % fullscene_pca_model.explained_variance_ratio_.cumsum())

Cumulative explained variance: [0.33318954 0.45449394 0.55398019 0.62720098 0.67315395 0.71082232]


### Plotting of the embeding

`plot_embedding` plots each questionnaire's response when projected along the first 2 eigenvectors. Responses' colours correspond to the clips.

In [17]:
plot_embedding(fullscene_pca, fullscene_labels, fullscene_means_pca, fullscene_means.index, title="'Fullscene' dataset projection onto its PCA space", three_d=False)
plot_embedding(skel_pca, skel_labels, skel_means_pca, skel_means.index, title="'Skeletal' dataset projection onto its PCA space", three_d=False) 

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

For each clip, we can then plot the distance between its embedding based on fullscene ratings vs its embedding based on skeletal data only.

In [18]:
plot_compare_embeddings(skel_means_pca, fullscene_means_pca, skel_means.index, title="O: skeletal, X: fullscene", three_d=False)
plot_compare_embeddings(skel_means_pca, fullscene_means_pca, skel_means.index, title="O: skeletal, X: fullscene", three_d=True)

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

Computing the actual distance between clips in the two conditions shows that they are generally quite far apart. A straightforward PCA embedding does not seem to be effective to evidence similarities between our 2 conditions.

In [19]:
distances_pca=pd.DataFrame(np.power(np.sum(np.power(skel_means_pca - fullscene_means_pca, 2), axis=1), 0.5), index=skel_means.index, columns=["distance_pca"])
show_heatmap(distances_pca, cmap="summer_r")

,distance_pca
clipId,
01,5.8938
02,0.904112
03,1.73459
04,4.33966
05,1.2738
06,2.41302
07,2.67892
08,1.57773
09,1.52314


To answer the question: *does a PCA evidence common latent factors between our 2 conditions?*, we compute a PCA model *based on the skeletal data*, and compare the resulting PCA components with the ones found with the fullscene data.

We observe that the resulting loadings look very different.

In [20]:
# plot of PCA components, with fullscene vs skeleton components side-by-side


skel_pca_model=PCA(n_components=nb_components).fit(skel)

skel_pca = skel_pca_model.transform(skel)
skel_means_pca = skel_pca_model.transform(skel_means.values)



skel_pca_components = pd.DataFrame(skel_pca_model.components_,columns=columnsDiffSum).T
fullscene_pca_components = pd.DataFrame(fullscene_pca_model.components_,columns=columnsDiffSum).T

# merge PCA components into one dataframe, skel and fullscene side-by-side
pca_components=pd.concat([skel_pca_components, fullscene_pca_components], keys=["skel", "fullscene"], axis=1)
pca_components=pca_components.swaplevel(0,1,1).sort_index(1)


show_heatmap(pca_components[abs(pca_components)>0.2], m=-0.6, M=0.6)


/home/s-lemaignan/.local/lib/python3.6/site-packages/matplotlib/colors.py:496: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


## LDA

We perform the LDA *on top of the PCA* as LDA typically requires O > 3 F, with O the nb of observations and F the nb of features (here, we have ~26 observations for originally 22 questions). Using the PCA as a dimensionality reduction tool, we bring down the number of degrees of freedom to 6.

In [21]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda_nb_components = 4

fullscene_lda_model = LinearDiscriminantAnalysis(n_components=lda_nb_components, solver='svd')
fullscene_lda_model.fit(fullscene_pca, fullscene_labels)

fullscene_lda = fullscene_lda_model.transform(fullscene_pca_model.transform(fullscene))
fullscene_means_lda = fullscene_lda_model.transform(fullscene_pca_model.transform(fullscene_means.values))

skel_lda = fullscene_lda_model.transform(fullscene_pca_model.transform(skel))
skel_means_lda = fullscene_lda_model.transform(fullscene_pca_model.transform(skel_means.values))


Attention: the variance explained by the LDA transformation is the variance in the *PCA* space, not in the original 22-D space of the questionnaire!

In [22]:
print("Cumulative explained variance by LDA: %s" % fullscene_lda_model.explained_variance_ratio_.cumsum())

Cumulative explained variance by LDA: [0.44355966 0.70321882 0.849057   0.91358403]


When projected in the PDA space, the clips in condition *skeleton* vs *fullscene* are much closer to one another.

In [23]:
distances_lda=pd.DataFrame(np.power(np.sum(np.power(skel_means_lda - fullscene_means_lda, 2), axis=1), 0.5), index=skel_means.index, columns=["distance_lda"])

distances = pd.concat([distances_pca, distances_lda], axis=1)
show_heatmap(distances, cmap="summer_r")


,distance_pca,distance_lda
clipId,,
01,5.8938,1.96528
02,0.904112,0.370138
03,1.73459,1.04416
04,4.33966,2.45472
05,1.2738,0.48901
06,2.41302,0.683214
07,2.67892,1.29882
08,1.57773,0.703964
09,1.52314,0.619488


In [24]:
plot_embedding(fullscene_lda, fullscene_labels,fullscene_means_lda, fullscene_means.index, title="LDA embedding, fullscene data", three_d=False)
plot_embedding(skel_lda, skel_labels,skel_means_lda, skel_means.index, title="LDA embedding, skeletal data", three_d=False)

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

In [25]:
plot_compare_embeddings(skel_means_lda, fullscene_means_lda, fullscene_means.index,three_d=False)
plot_compare_embeddings(skel_means_lda, fullscene_means_lda, skel_means.index,three_d=True)

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

To answer the question: *does a PCA evidence common latent factors between our 2 conditions?*, we can again create a new LDA model for the skeletal data, and print out the LDA components for *fullscene* vs *skeleton* side-by-side.

We observe that they still look very different.

In [26]:
# compute as well a LDA model from the skeletal PCA to compare components with fullscene
skel_lda_model = LinearDiscriminantAnalysis(n_components=lda_nb_components, solver='svd')
skel_lda_model.fit(skel_pca, skel_labels)


fullscene_lda_components = pd.DataFrame(fullscene_lda_model.scalings_[:,:lda_nb_components].T).T
skel_lda_components = pd.DataFrame(skel_lda_model.scalings_[:,:lda_nb_components].T).T

# merge PCA components into one dataframe, skel and fullscene side-by-side
lda_components=pd.concat([fullscene_lda_components, skel_lda_components], keys=["fullscene", "skel"], axis=1)
lda_components=lda_components.swaplevel(0,1,1).sort_index(1)

show_heatmap(lda_components[abs(lda_components)>0.2], m=-0.6, M=0.6)

/home/s-lemaignan/.local/lib/python3.6/site-packages/matplotlib/colors.py:496: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


By multiplying the LDA loadings matrix with the PCA loadings matrix, we can compute the LDA loadings in terms of the original questions asked to the participants.

In [27]:
lda_fullscene_loadings=pd.DataFrame(np.dot(fullscene_pca_components, fullscene_lda_components), index=pca_components.index, columns=["LDA component %d" % (i+1) for i in range(lda_nb_components)])
lda_skel_loadings=pd.DataFrame(np.dot(skel_pca_components, skel_lda_components), index=pca_components.index, columns=["LDA component %d" % (i+1) for i in range(lda_nb_components)])
# merge loadings into one dataframe, skel and fullscene side-by-side
lda_loadings=pd.concat([lda_fullscene_loadings, lda_skel_loadings], keys=["fullscene","skel"], axis=1)
lda_loadings=lda_loadings.swaplevel(0,1,1).sort_index(1)

from scipy.stats import pearsonr

print("Pearson correlation between LDA components 'fullscene' vs 'skeletal'")
for i in range(1, lda_nb_components + 1):
    r, p=pearsonr(lda_loadings["LDA component %d" % i]["fullscene"].values, lda_loadings["LDA component %d" % i]["skel"].values)
    print("LDA component %d: r=%f, p=%f" % (i,r,p)) 


show_heatmap(lda_loadings[abs(lda_loadings)>=0.1], m=-0.6, M=0.6)

Pearson correlation between LDA components 'fullscene' vs 'skeletal'
LDA component 1: r=0.983706, p=0.000000
LDA component 2: r=-0.378594, p=0.056494
LDA component 3: r=-0.520672, p=0.006390
LDA component 4: r=-0.196703, p=0.335488


/home/s-lemaignan/.local/lib/python3.6/site-packages/matplotlib/colors.py:496: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


## Explorative Factor Analysis

The Python [factor_analyzer](https://factor-analyzer.readthedocs.io) module is a port of EFA from the R' `psych` package.

In [28]:
import factor_analyzer

rotation = 'promax'

nb_factors=3

fa_fullscene = factor_analyzer.FactorAnalyzer()
fa_fullscene.analyze(fullscene_ratings_df, nb_factors, rotation=rotation)
fullscene_loadings=fa_fullscene.loadings

fa_skel = factor_analyzer.FactorAnalyzer()
fa_skel.analyze(skel_ratings_df, nb_factors, rotation=rotation)
skel_loadings=fa_skel.loadings

Comparing the loadings for the *fullscene* vs the *skeletal* only data show that the first two factors are highly correlated. **This shows that, using factor analysis, we have uncovered latent constructs that are used by participants to describe the clips in both *fullscene* and *skeletal-only* conditions**.

In [29]:
# merge loadings into one dataframe, skel and fullscene side-by-side
loadings=pd.concat([fullscene_loadings, skel_loadings], keys=["fullscene","skel"], axis=1)
loadings=loadings.swaplevel(0,1,1).sort_index(1)

from scipy.stats import pearsonr

print("Pearson correlation between factors 'fullscene' vs 'skeletal'")
for i in range(1, nb_factors+1):
    r, p=pearsonr(loadings["Factor%d" % i]["fullscene"].values, loadings["Factor%d" % i]["skel"].values)
    print("Factor %d: r=%f, p=%f" % (i,r,p)) 
    
    
show_heatmap(loadings[abs(loadings)>=0.3])


Pearson correlation between factors 'fullscene' vs 'skeletal'
Factor 1: r=0.938501, p=0.000000
Factor 2: r=0.865091, p=0.000000
Factor 3: r=0.820778, p=0.000000


/home/s-lemaignan/.local/lib/python3.6/site-packages/matplotlib/colors.py:496: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


constructs=["Dominant", "Cooperative", "Competitive", "Friendly", "Aggressive", "Engaged", "Fearful", "Sad", "Content", "Angry", "Amused"]

loadings2 = pd.DataFrame()

for c in constructs:
    loadings2[c + ": diff."] = abs(loadings.T["left" + c] - loadings.T["right" + c])
    loadings2[c + ": mag."] = np.amax([abs(loadings.T["left" + c]), abs(loadings.T["right" + c])], axis=0)

show_heatmap(loadings2.T, cmap="summer_r")


In [30]:
fa_skel.get_factor_variance()


,Factor1,Factor2,Factor3
SS Loadings,4.805225,3.677328,3.590071
Proportion Var,0.184816,0.141436,0.138080
Cumulative Var,0.184816,0.326252,0.464332


In [31]:
fa_fullscene.get_factor_variance()

,Factor1,Factor2,Factor3
SS Loadings,3.960846,3.993708,3.747214
Proportion Var,0.152340,0.153604,0.144124
Cumulative Var,0.152340,0.305944,0.450068


### EFA embeddings

We can use the EFA space as a 'better' space to represent our clips, where the latent, composite constructs correspond to the main axis:

In [32]:
nb_of_factors=6
fullscene_efa = np.dot(fullscene,fullscene_loadings.values[:,:nb_of_factors])
fullscene_means_efa = np.dot(fullscene_means,fullscene_loadings.values[:,:nb_of_factors])
skel_efa = np.dot(skel,fullscene_loadings.values[:,:nb_of_factors])
skel_means_efa = np.dot(skel_means,fullscene_loadings.values[:,:nb_of_factors])

skel_pure_efa = np.dot(skel,skel_loadings.values[:,:nb_of_factors])
skel_pure_means_efa = np.dot(skel_means,skel_loadings.values[:,:nb_of_factors])


plot_embedding(fullscene_efa, fullscene_labels,fullscene_means_efa, fullscene_means.index, title="EFA-space embedding of the fullscene data", three_d=True)
plot_embedding(skel_efa, skel_labels,skel_means_efa, skel_means.index, title="EFA-space embedding of the skeletal data (EFA on fullscene data)", three_d=False)
plot_embedding(skel_pure_efa, skel_labels,skel_pure_means_efa, skel_means.index, title="EFA-space embedding of the skeletal data (EFA on skel data)", three_d=False)


'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.


Interestingly, even if the EFA factors are quite similar, the distances between same clips in fullscene vs skeletal data are higher in the EFA space compared to the PCA or LDA space:

In [33]:
distances_efa=pd.DataFrame(np.power(np.sum(np.power(skel_means_efa - fullscene_means_efa, 2), axis=1), 0.5), index=skel_means.index, columns=["distance_efa"])

distances = pd.concat([distances_pca, distances_lda, distances_efa], axis=1)
print("Mean distances:\n%s" % distances.mean(axis=0))
show_heatmap(distances, cmap="summer_r")


Mean distances:
distance_pca    2.404621
distance_lda    0.963087
distance_efa    3.789820
dtype: float64


,distance_pca,distance_lda,distance_efa
clipId,,,
01,5.8938,1.96528,9.99353
02,0.904112,0.370138,1.83519
03,1.73459,1.04416,2.13302
04,4.33966,2.45472,4.80401
05,1.2738,0.48901,2.23191
06,2.41302,0.683214,4.68567
07,2.67892,1.29882,4.63798
08,1.57773,0.703964,1.82828
09,1.52314,0.619488,2.49492


# Clustering



Based on the distance measurements, the LDA space seems to be the best to cluster our clips.
We can then attempt to cluster our 20 clips into 'groups' of similar clips (based on the latent constructs):

In [34]:
from sklearn.cluster import KMeans

# kMeans clustering after projecting our clips in the EFA-space
fullscene_clustering_data=fullscene_means_lda

nb_clusters=5

fullscene_kmeans_model = KMeans(n_clusters=nb_clusters, random_state=0).fit(fullscene_clustering_data)
fullscene_kmeans = fullscene_kmeans_model.predict(fullscene_clustering_data)

plot_embedding(fullscene_clustering_data,fullscene_means.index,clusters=fullscene_kmeans, three_d=True)

pd.DataFrame(fullscene_kmeans, index=fullscene_means.index, columns=["cluster #"]).sort_values(by="cluster #")

,cluster #
clipId,
01,0
17,0
12,0
10,0
03,0
04,0
06,1
19,1
11,1


We should be able to infer the semantics of the 3 first EFA factors.


We can then try to predict in which cluster the clips would end up, using only the ratings from the skeletal videos:

In [35]:
skel_means_efa.shape

(20, 3)

In [36]:
skel_kmeans= fullscene_kmeans_model.predict(skel_means_lda)

plot_embedding(fullscene_means_lda,skel_means.index, clusters=skel_kmeans, three_d=False)

diff=pd.DataFrame(fullscene_kmeans-skel_kmeans,index=skel_means.index)
print("%d skeleton clips out of %d (%.1f%%) are predicted to fall into the same cluster as their 'fullscene' counterpart." % (diff[diff==0].count(), skel_kmeans.size, diff[diff==0].count() * 100. / skel_kmeans.size))

clusters_kripp=pd.DataFrame([fullscene_kmeans, skel_kmeans, fullscene_kmeans==skel_kmeans,krippendorff_df[["alpha(fullscene)", "alpha(skel)"]].std(axis=1).astype(float), krippendorff_df[["alpha(fullscene)", "alpha(skel)"]].mean(axis=1).astype(float), krippendorff_df["alpha(fullscene)"], krippendorff_df["alpha(skel)"], ],index=["fullscene clusters", "skel clusters", "same", "kripp alpha std", "kripp alpha mean", "alpha(fullscene)", "alpha(skel)"],columns=skel_means.index).T.sort_values(by="kripp alpha mean")
clusters_kripp

13 skeleton clips out of 20 (65.0%) are predicted to fall into the same cluster as their 'fullscene' counterpart.


,fullscene clusters,skel clusters,same,kripp alpha std,kripp alpha mean,alpha(fullscene),alpha(skel)
clipId,,,,,,,
09,4,4,True,0.0280486,0.0776562,0.0974896,0.0578229
07,4,4,True,0.102169,0.163664,0.0914201,0.235908
16,4,4,True,0.0818972,0.214314,0.156404,0.272224
17,0,3,False,0.0633495,0.227733,0.272528,0.182938
02,4,4,True,0.0557975,0.230434,0.19098,0.269889
10,0,4,False,0.219404,0.241284,0.396426,0.0861424
11,1,4,False,0.0326347,0.257414,0.280491,0.234338
13,3,3,True,0.10236,0.261617,0.333996,0.189237
05,2,2,True,0.0314335,0.305662,0.327889,0.283435


Is there a correlation between 'same clusters' and Krippendorf agreement (ie, consistency of ratings for a given clip)? No...

In [37]:
print("Mean Krippendorf, same cluster: %f" % clusters_kripp[clusters_kripp["same"] == True]["kripp alpha mean"].mean())
print("Std Krippendorf, same cluster: %f" % clusters_kripp[clusters_kripp["same"] == True]["kripp alpha mean"].std())
print("Mean Krippendorf, diff cluster: %f" % clusters_kripp[clusters_kripp["same"] == False]["kripp alpha mean"].mean())
print("Std Krippendorf, diff cluster: %f" % clusters_kripp[clusters_kripp["same"] == False]["kripp alpha mean"].std())

Mean Krippendorf, same cluster: 0.289202
Std Krippendorf, same cluster: 0.096639
Mean Krippendorf, diff cluster: 0.300357
Std Krippendorf, diff cluster: 0.056100


# Classification



Training a multi-label classifier on our clips

In [277]:
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer



training_ground_truth = { '01': ['Aggressive'],
                         '02': ['Excited', 'Aggressive', 'Aimless'],
                         '03': ['Excited', 'Fun'],
                         '04': ['Cooperative'],
                         '05': ['Bored', 'Aimless'],
                         '06': ['Cooperative'],
                         '07': ['Dominant'],
                         '08': ['Bored', 'Fun'],
                         '09': ['Cooperative'],
                         '10': ['Cooperative', 'Dominant'],
                         '11': ['Cooperative', 'Dominant'],
                         '12': ['Aggressive', 'Aimless'],
                         '13': ['Excited', 'Aggressive', 'Aimless'],
                         '14': ['Aggressive'],
                         '15': ['Dominant'],
                         '16': ['Cooperative', 'Dominant'],
                         '17': ['Excited', 'Aggressive'],
                         '18': ['Aggressive', 'Dominant'],
                         '19': ['Dominant'],
                         '20': ['Excited']}

mlb = MultiLabelBinarizer()
mlb.fit(training_ground_truth.values())

def datasets(training=df, testing=None, cols=allQuestions, test_size=0.2, use_clip_id_as_label=False, random_labels=False):
    """Returns a training dataset and training labels, and a testing dataset and testing labels.
    
    If testing is None, it randomly splits the training dataframe (at test_size).
    """


    if testing is None:
        
        if use_clip_id_as_label:
            labels = list(training["clipId"].map(int))
        else:
            labels = []
            for id in training["clipId"]:
                labels.append(training_ground_truth[id])

        data = training[cols].values

        training_data, testing_data, training_labels, testing_labels = train_test_split(data, labels, test_size=test_size, random_state=int(time.time()))

        if not use_clip_id_as_label:
            
            training_labels, testing_labels = mlb.transform(training_labels), mlb.transform(testing_labels)
            
            if random_labels:
                for labels in testing_labels:
                    np.random.shuffle(labels)                 
                np.random.shuffle(testing_labels)             
            

        return training_data, testing_data, training_labels, testing_labels
    
    else:
        
        if use_clip_id_as_label:
            training_labels = list(training["clipId"].map(int))
            testing_labels = list(testing["clipId"].map(int))
        else:
            labels = []
            for id in training["clipId"]:
                labels.append(training_ground_truth[id])

            training_labels = mlb.transform(labels)

            labels = []
            for id in testing["clipId"]:
                labels.append(training_ground_truth[id])

            testing_labels = mlb.transform(labels)

            if random_labels:
                if random_labels:
                    for labels in testing_labels:
                        np.random.shuffle(labels)                 
                    np.random.shuffle(testing_labels) 

        
        training_data = training[cols].values
        testing_data = testing[cols].values

        return training_data, testing_data, training_labels, testing_labels

In [278]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import ExtraTreeClassifier

def train(training_data, training_labels):
    
    #clf = RandomForestClassifier()
    #clf = KNeighborsClassifier(n_neighbors=3)
    #clf = ExtraTreeClassifier(random_state=0)
    
    layers_nb = 3
    layer_size = 20

    layers = (layer_size, ) * layers_nb
    print("Training a MLP classifier, layers: %s..." % str(layers))

    clf = MLPClassifier(hidden_layer_sizes=layers, activation='relu', max_iter=1000, solver="lbfgs")
    clf.fit(training_data, training_labels)
    
    return clf

def predict(clf, testing_data, inverse_transform_labels=True):
    p = clf.predict(testing_data)
    if inverse_transform_labels:
        return mlb.inverse_transform(p) 
    else:
        return p

In [307]:
def run_classification(training, testing=None, cols=allQuestions, use_clip_id_as_label=False, random_labels=False, iterations=50):
    """
    Metrics for multi-label classification coming form Sorower, Mohammad S. "A literature survey on algorithms for multi-label learning." Oregon State University, Corvallis (2010).
    """
    
    mean_score_exact = 0
    mean_accuracy = 0
    mean_precision = 0
    mean_recall = 0
    mean_f1_measure = 0

    for x in range(iterations):
        print("Round %d/%d" % (x+1, iterations))
        training_data, testing_data, training_labels, testing_labels = datasets(training=training, testing=testing, cols=allQuestions, use_clip_id_as_label=use_clip_id_as_label, random_labels=random_labels)
        
        if iterations == 1:
            print("Shape of training data: %s" % str(training_data.shape))
            print("Shape of testing data: %s" % str(testing_data.shape))

        clf = train(training_data, training_labels)

        #mean_score_exact += clf.score(testing_data, testing_labels)

        pred_labels = predict(clf, testing_data, inverse_transform_labels = not use_clip_id_as_label)


        
        at_least_one = 0
        at_least_one_no_incorrect = 0
        
        if not use_clip_id_as_label:
            
            nb_classes = len(mlb.classes_)
            
            testing_labels = mlb.inverse_transform(testing_labels)
            
            exact = 0
            accuracy = 0
            precision = 0
            recall = 0
            f1_measure = 0
            
            for actual, pred in zip(testing_labels, pred_labels):
                
                pred = set(pred)
                actual = set(actual)
                
                if len(pred) == 0: continue
                    
                if pred == actual:
                    #print("%s <-> %s" % (actual, pred))
                    exact += 1
                    
                intersection = pred.intersection(actual)
                union = pred.union(actual)

                accuracy += float(len(intersection)) / len(union)
                precision += float(len(intersection)) / len(pred)
                recall += float(len(intersection)) / len(actual)
                f1_measure += 2 * float(len(intersection)) / (len(pred) + len(actual))
                
            
            mean_score_exact += float(exact) / len(testing_labels)
            mean_accuracy += accuracy / len(testing_labels)
            mean_precision += precision / len(testing_labels)
            mean_recall += recall / len(testing_labels)
            mean_f1_measure += f1_measure / len(testing_labels)
            
        else: # use_clip_id_as_label = True
            # does not make much sense as at_least_one & at_least_one_no_incorrect are the same as 'exact'
            pass


    exact_match_ratio = mean_score_exact/iterations
    accuracy = mean_accuracy/iterations
    precision = mean_precision/iterations
    recall = mean_recall/iterations
    f1_measure = mean_f1_measure/iterations
    
    return exact_match_ratio, accuracy, precision, recall, f1_measure

In [306]:
exact_match_ratio, accuracy, precision, recall, f1_measure = run_classification(fullscene_df, iterations=20)
print("Fullscene - 80%%/20%% split - multi-label - Exact match ratio (MR): %f%%" % exact_match_ratio)
print("Fullscene - 80%%/20%% split - multi-label - Accuracy: %f%%" % accuracy)
print("Fullscene - 80%%/20%% split - multi-label - Precision: %f%%" % precision)
print("Fullscene - 80%%/20%% split - multi-label - Recall: %f%%" % recall)
print("Fullscene - 80%%/20%% split - multi-label - F1-mesure: %f%%" % f1_measure)


Round 1/20
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Training a MLP classifier, layers: (20, 20, 20)...
Round 2/20
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Training a MLP classifier, layers: (20, 20, 20)...
Round 3/20
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Training a MLP classifier, layers: (20, 20, 20)...
Round 4/20
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Training a MLP classifier, layers: (20, 20, 20)...
Round 5/20
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Training a MLP classifier, layers: (20, 20, 20)...
Round 6/20
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Training a MLP classifier, layers: (20, 20, 20)...
Round 7/20
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Training a MLP classifier, layers: (20, 20, 20)...
Round 8/20
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Training a MLP classifier, 

In [308]:
exact_match_ratio, accuracy, precision, recall, f1_measure = run_classification(fullscene_df, random_labels=True, iterations=20)

print("Fullscene - 80%%/20%% split - multi-label - Exact match ratio (MR): %f%%" % exact_match_ratio)
print("Fullscene - 80%%/20%% split - multi-label - Accuracy: %f%%" % accuracy)
print("Fullscene - 80%%/20%% split - multi-label - Precision: %f%%" % precision)
print("Fullscene - 80%%/20%% split - multi-label - Recall: %f%%" % recall)
print("Fullscene - 80%%/20%% split - multi-label - F1-mesure: %f%%" % f1_measure)



Round 1/20
Training a MLP classifier, layers: (20, 20, 20)...
Round 2/20
Training a MLP classifier, layers: (20, 20, 20)...
Round 3/20
Training a MLP classifier, layers: (20, 20, 20)...
Round 4/20
Training a MLP classifier, layers: (20, 20, 20)...
Round 5/20
Training a MLP classifier, layers: (20, 20, 20)...
Round 6/20
Training a MLP classifier, layers: (20, 20, 20)...
Round 7/20
Training a MLP classifier, layers: (20, 20, 20)...
Round 8/20
Training a MLP classifier, layers: (20, 20, 20)...
Round 9/20
Training a MLP classifier, layers: (20, 20, 20)...
Round 10/20
Training a MLP classifier, layers: (20, 20, 20)...
Round 11/20
Training a MLP classifier, layers: (20, 20, 20)...
Round 12/20
Training a MLP classifier, layers: (20, 20, 20)...
Round 13/20
Training a MLP classifier, layers: (20, 20, 20)...
Round 14/20
Training a MLP classifier, layers: (20, 20, 20)...
Round 15/20
Training a MLP classifier, layers: (20, 20, 20)...
Round 16/20
Training a MLP classifier, layers: (20, 20, 20)...
R

In [273]:
exact, _, _, _, _ = run_classification(fullscene_df, use_clip_id_as_label=True, iterations=25)

print("Fullscene - 80%%/20%% split - label=clip_id: %f%%" % exact)

Round 1/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 2/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 3/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 4/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 5/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 6/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 7/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 8/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 9/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 10/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 11/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 12/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 13/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 14/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 15/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 16/25
Training a MLP classifier, layers: (20, 20, 20)...
R

In [310]:
exact_match_ratio, accuracy, precision, recall, f1_measure = run_classification(fullscene_df, testing=skel_df, iterations=25)

print("Fullscene vs skel - multi-label  - Exact match ratio (MR): %f%%" % exact_match_ratio)
print("Fullscene vs skel - multi-label  -  Accuracy: %f%%" % accuracy)
print("Fullscene vs skel - multi-label  -  Precision: %f%%" % precision)
print("Fullscene vs skel - multi-label  -  Recall: %f%%" % recall)
print("Fullscene vs skel - multi-label  -  F1-mesure: %f%%" % f1_measure)

Round 1/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 2/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 3/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 4/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 5/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 6/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 7/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 8/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 9/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 10/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 11/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 12/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 13/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 14/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 15/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 16/25
Training a MLP classifier, layers: (20, 20, 20)...
R

In [309]:
exact_match_ratio, accuracy, precision, recall, f1_measure = run_classification(fullscene_df, testing=skel_df, random_labels=True, iterations=25)

print("Fullscene vs skel - multi-label  - Exact match ratio (MR): %f%%" % exact_match_ratio)
print("Fullscene vs skel - multi-label  - Accuracy: %f%%" % accuracy)
print("Fullscene vs skel - multi-label  - Precision: %f%%" % precision)
print("Fullscene vs skel - multi-label  - Recall: %f%%" % recall)
print("Fullscene vs skel - multi-label  - F1-mesure: %f%%" % f1_measure)

Round 1/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 2/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 3/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 4/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 5/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 6/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 7/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 8/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 9/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 10/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 11/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 12/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 13/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 14/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 15/25
Training a MLP classifier, layers: (20, 20, 20)...
Round 16/25
Training a MLP classifier, layers: (20, 20, 20)...
R

In [39]:
clf_clusters = MLPClassifier(hidden_layer_sizes=(20,20,20), activation='relu', max_iter=1000, solver="lbfgs")

# trying with skel, skel_pca, skel_lda do not lead to any clear improvements
training_set = fullscene_lda

training_labels = []

for id in fullscene_df["clipId"]:
    training_labels.append(fullscene_kmeans[int(id)-1])

testing_labels = []

for id in range(20):
    testing_labels.append(fullscene_kmeans[int(id)-1])

testing_set = skel_means_lda


clf_clusters.fit(training_set, training_labels)

print("Full scene clusters:    " + str(fullscene_kmeans))
print("Predicted clusters skel:" + str(clf_clusters.predict(testing_set)))

clf_clusters.score(testing_set, testing_labels)

Full scene clusters:    [0 4 0 0 2 1 4 2 4 0 1 0 3 1 1 4 0 3 1 1]
Predicted clusters skel:[4 3 0 4 1 1 4 1 1 1 0 3 0 1 3 3 0 1 4 4]


0.1

In [40]:
skel_df[allQuestions].values[5]

array([ 0,  4,  0,  4,  0, -4,  0,  2,  0, -4,  0,  0,  0,  4,  0,  4,  0,
       -4,  0,  4,  0, -4,  0, -4,  0, -4,  0, -4,  0, -4])

In [41]:
skel_means_efa

array([[ 6.55335591, -2.68223925, -2.79376632],
       [ 6.61346505, -5.59856848, -2.74701388],
       [ 4.3998907 , -7.6569428 , -5.15983324],
       [ 5.70003862, -7.97038767, -1.93302849],
       [ 7.27455317, -6.20924178,  5.3369516 ],
       [ 4.99022376, -9.17887662, -2.00672494],
       [ 8.0162446 , -6.29792367, -1.54210744],
       [ 6.65395516, -5.93712916,  6.1430186 ],
       [ 7.79009567, -3.51582411,  1.77581716],
       [ 7.44148387, -4.6910501 ,  1.31075797],
       [ 5.78674697, -5.10560686, -2.30371266],
       [ 5.84307949, -7.45426663, -3.20475794],
       [ 7.62710248, -3.33245345, -2.69979882],
       [ 6.48693611, -7.64446231, -3.00639006],
       [ 9.34816385, -4.37153493,  0.68814499],
       [ 6.71684805, -7.27969492, -2.2385575 ],
       [ 7.73563156, -3.72555758, -1.78349253],
       [10.02386103, -3.03024193,  0.32503986],
       [ 4.16916871, -7.6405634 , -2.44691327],
       [ 6.55946842, -6.55764337, -1.82424428]])

Using a SVM classifier, we can try to improve our predictions:

In [42]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')

# trying with skel, skel_pca, skel_lda do not lead to any clear improvements
training_set = fullscene_means_lda
training_labels = fullscene_kmeans

testing_set = skel_means_lda
# Critically, we are testing with the *fullscene_kmeans* as we want to check whether 
# we predict the same clusters as with the fullscene stimuli.
testing_labels = fullscene_kmeans

clf.fit(training_set, training_labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [43]:
for p, l in zip(clf.predict(testing_set), testing_labels):
    print("%s (should be %s)" % (p,l))
print("SVM: %.1f%% successful prediction out of %d tested clips" % (clf.score(testing_set, testing_labels) * 100, len(testing_labels)))

4 (should be 0)
4 (should be 4)
0 (should be 0)
4 (should be 0)
2 (should be 2)
1 (should be 1)
4 (should be 4)
2 (should be 2)
4 (should be 4)
4 (should be 0)
4 (should be 1)
4 (should be 0)
4 (should be 3)
1 (should be 1)
4 (should be 1)
4 (should be 4)
4 (should be 0)
4 (should be 3)
1 (should be 1)
1 (should be 1)
SVM: 55.0% successful prediction out of 20 tested clips


In [44]:
from sklearn.metrics import confusion_matrix

cnf_matrix = confusion_matrix(testing_labels, clf.predict(testing_set))
plot_confusion_matrix(cnf_matrix, classes=pd.unique(testing_labels))

Confusion matrix, without normalization
